![Microsoft](https://raw.githubusercontent.com/microsoft/azuredatastudio/master/src/sql/media/microsoft-small-logo.png)
 
## Create Azure Kubernetes Service cluster and deploy SQL Server 2019 Big Data Cluster
 
This notebook walks through the process of creating a new Azure Kubernetes Service cluster first, and then deploys a <a href="https://docs.microsoft.com/sql/big-data-cluster/big-data-cluster-overview?view=sqlallproducts-allversions">SQL Server 2019 Big Data Cluster</a> on the newly created AKS cluster.

In [1]:
import json,sys,os
def run_command(command):
    print("Executing: " + command)
    !{command}
    if _exit_code != 0:
        sys.exit(f'Command execution failed with exit code: {str(_exit_code)}.\n\t{command}\n')
    print(f'Successfully executed: {command}')

### **Set variables**
Generated by Azure Data Studio using the values collected in the Deploy Big Data Cluster wizard

### **Set password**

In [0]:
mssql_password = os.environ["AZDATA_NB_VAR_BDC_ADMIN_PASSWORD"]

### **Login to Azure**

This will open a web browser window to enable credentials to be entered. If this cells is hanging forever, it might be because your Web browser windows is waiting for you to enter your Azure credentials!


In [5]:
run_command(f'az login')


### **Set active Azure subscription**

In [6]:
if azure_subscription_id != "":
    run_command(f'az account set --subscription {azure_subscription_id}')
else:
    print('Using the default Azure subscription', {azure_subscription_id})
run_command(f'az account show')

### **Create Azure resource group**

In [7]:
run_command(f'az group create --name {azure_resource_group} --location {azure_region}')

### **Create AKS cluster**

In [8]:
run_command(f'az aks create --name {aks_cluster_name} --resource-group {azure_resource_group} --generate-ssh-keys --node-vm-size {azure_vm_size} --node-count {azure_vm_count}')

### **Set the new AKS cluster as current context**

In [9]:
run_command(f'az aks get-credentials --resource-group {azure_resource_group} --name {aks_cluster_name} --admin --overwrite-existing')

### **Create deployment configuration files**

In [10]:
mssql_target_profile = 'ads-bdc-custom-profile'
if not os.path.exists(mssql_target_profile):
    os.mkdir(mssql_target_profile)
bdcJsonObj = json.loads(bdc_json)
controlJsonObj = json.loads(control_json)
bdcJsonFile = open(f'{mssql_target_profile}/bdc.json', 'w')
bdcJsonFile.write(json.dumps(bdcJsonObj, indent = 4))
bdcJsonFile.close()
controlJsonFile = open(f'{mssql_target_profile}/control.json', 'w')
controlJsonFile.write(json.dumps(controlJsonObj, indent = 4))
controlJsonFile.close()
print(f'Created deployment configuration folder: {mssql_target_profile}')

### **Create SQL Server 2019 Big Data Cluster**

In [11]:
print (f'Creating SQL Server 2019 Big Data Cluster: {mssql_cluster_name} using configuration {mssql_target_profile}')
os.environ["ACCEPT_EULA"] = 'yes'
os.environ["AZDATA_USERNAME"] = mssql_username
os.environ["AZDATA_PASSWORD"] = mssql_password
if os.name == 'nt':
    print(f'If you don\'t see output produced by azdata, you can run the following command in a terminal window to check the deployment status:\n\tkubectl get pods -n {mssql_cluster_name} ')
run_command(f'azdata bdc create -c {mssql_target_profile}')